In [ ]:
%%capture

%cd ..

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
# Two lists of sentences
sentences1 = [
    "The cat sits outside",
    "A man is playing guitar",
    "The new movie is awesome",
]
sentences2 = [
    "The dog plays in the garden",
    "A woman watches TV",
    "The new movie is so great",
]

In [ ]:
# Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)
print(embeddings1.shape)
assert embeddings1.shape == embeddings2.shape

torch.Size([3, 384])


In [ ]:
# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cosine_scores.shape

torch.Size([3, 3])

In [ ]:
# Output the pairs with their score
for i in range(len(sentences1)):
    print(
        "{} \t\t {} \t\t Score: {:.4f}".format(
            sentences1[i], sentences2[i], cosine_scores[i][i]
        )
    )

The cat sits outside 		 The dog plays in the garden 		 Score: 0.2838
A man is playing guitar 		 A woman watches TV 		 Score: -0.0327
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939


In [ ]:
cosine_scores

tensor([[ 0.2838,  0.1310, -0.0029],
        [ 0.2277, -0.0327, -0.0136],
        [ 0.0543, -0.0502,  0.8939]])

Rank `sentences1` by most similar sentences in `sentences2`.

In [ ]:
import torch

# do the ranking
ranks = torch.sort(cosine_scores, dim=-1, descending=True)
ranks


torch.return_types.sort(
values=tensor([[ 0.2838,  0.1310, -0.0029],
        [ 0.2277, -0.0136, -0.0327],
        [ 0.8939,  0.0543, -0.0502]]),
indices=tensor([[0, 1, 2],
        [0, 2, 1],
        [2, 0, 1]]))

In [ ]:
diff_with_top = ranks.values[:, 0, None] - ranks.values

for i, (val, idx, diff) in enumerate(zip(*ranks, diff_with_top)):
    print(f"Most similar sentences to {sentences1[i]!r}:")
    for v, j in zip(val, idx):
        print((sentences2[j], v))
    print(f"Delta similarity with rank 1: {diff}")
    print()

Most similar sentences to 'The cat sits outside':
('The dog plays in the garden', tensor(0.2838))
('A woman watches TV', tensor(0.1310))
('The new movie is so great', tensor(-0.0029))
Delta similarity with rank 1: tensor([0.0000, 0.1527, 0.2866])

Most similar sentences to 'A man is playing guitar':
('The dog plays in the garden', tensor(0.2277))
('The new movie is so great', tensor(-0.0136))
('A woman watches TV', tensor(-0.0327))
Delta similarity with rank 1: tensor([0.0000, 0.2413, 0.2604])

Most similar sentences to 'The new movie is awesome':
('The new movie is so great', tensor(0.8939))
('The dog plays in the garden', tensor(0.0543))
('A woman watches TV', tensor(-0.0502))
Delta similarity with rank 1: tensor([0.0000, 0.8396, 0.9441])

